In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame('image_train_data/')

[INFO] This non-commercial license of GraphLab Create is assigned to nnamdijr@gmail.comand will expire on September 24, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2460 - Server binary: C:\Anaconda\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Nnamdi\AppData\Local\Temp\graphlab_server_1446187804.log.0
[INFO] GraphLab Server Version: 1.6.1


In [5]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [9]:
dog = image_train[image_train['label'] == 'dog']
cat = image_train[image_train['label'] == 'cat']
auto = image_train[image_train['label'] == 'automobile']
bird = image_train[image_train['label'] == 'bird']

In [11]:
dog_model = graphlab.nearest_neighbors.create(dog,features=['deep_features'],
                                             label='id')
cat_model = graphlab.nearest_neighbors.create(cat,features=['deep_features'],
                                             label='id')
auto_model = graphlab.nearest_neighbors.create(auto,features=['deep_features'],
                                             label='id')
bird_model = graphlab.nearest_neighbors.create(bird,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.


In [12]:
image_test = graphlab.SFrame('image_test_data/')

In [17]:
cat_knn = cat_model.query(image_test[0:1])

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 29.003ms     |
PROGRESS: | Done         |         | 100         | 232.013ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [18]:
dog_knn = dog_model.query(image_test[0:1])

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 34.004ms     |
PROGRESS: | Done         |         | 100         | 186.015ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [19]:
dog_knn['distance'].mean()

37.77071136184156

In [20]:
cat_knn['distance'].mean()

36.15573070978294

In [21]:
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_auto = image_test[image_test['label'] == 'automobile']
image_test_bird = image_test[image_test['label'] == 'bird']

In [23]:
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
dog_auto_neighbors = auto_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 1.18s        |
PROGRESS: | Done         | 509000  | 100         | 1.33s        |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 1.14s        |
PROGRESS: 

In [25]:
dog_distances = graphlab.SFrame({'dog-dog': dog_dog_neighbors['distance'],'dog-cat': dog_cat_neighbors['distance']
                                ,'dog-auto': dog_auto_neighbors['distance'],'dog-bird': dog_bird_neighbors['distance']})

In [29]:
dog_distances['dog-cat'][0]

36.41960770675437

In [37]:
def is_dog_correct(row):
    if row['dog-dog'] < row['dog-cat'] and row['dog-dog'] < row['dog-auto'] and row['dog-dog'] < row['dog-bird']:
        return 1
    else:
        return 0

In [38]:
x = dog_distances.apply(is_dog_correct)

In [40]:
x.sum()

678L

In [41]:
image_test['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  4000 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+------+------+------+
| value | automobile | cat  | bird | dog  |
+-------+------------+------+------+------+
| count |    1000    | 1000 | 1000 | 1000 |
+-------+------------+------+------+------+


In [43]:
678/float(1000)

0.678

In [46]:
graphlab.canvas.set_target('ipynb')

In [47]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [53]:
y = get_images_from_ids(dog_knn)

In [55]:
y['image'][2].show()

In [54]:
y

id,image,label,deep_features,image_array
6094,Height: 32 Width: 32,dog,"[0.470533549786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[91.0, 98.0, 71.0, 138.0,123.0, 63.0, 135.0, ..."
13387,Height: 32 Width: 32,dog,"[0.366494178772, 0.0,0.0, 0.0, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
16976,Height: 32 Width: 32,dog,"[0.755595386028, 0.0,0.0, 0.0, 0.0, 0.0, ...","[16.0, 17.0, 11.0, 18.0,19.0, 13.0, 20.0, 21.0, ..."
35867,Height: 32 Width: 32,dog,"[0.305321395397, 0.0,0.0, 0.0, 0.0, 0.0, ...","[101.0, 93.0, 9.0, 93.0,88.0, 9.0, 90.0, 86.0, ..."
44603,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 11.2646656036, 0.0, ...","[8.0, 25.0, 9.0, 29.0,39.0, 22.0, 66.0, 75.0, ..."
